In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, DatasetDict
import torch

In [2]:
# Load your dataset
dataset = load_dataset("json", data_files="./arxiv_dataset.json")

In [3]:
# Load tokenizer
model_name = "gpt2-small"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

OSError: gpt2-small is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# Prepare text data
def preprocess_data(examples):
    return {"text": examples["title"] + " " + examples["abstract"]}
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["authors", "categories", "update_date"])

In [ ]:
# Tokenize the text data
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
tokenized_datasets = tokenized_dataset.map(tokenize_function, batched=True)


In [ ]:
# Remove the original text column
tokenized_datasets = tokenized_datasets.remove_columns(["text"])


In [ ]:
# Split the dataset
split_dataset = tokenized_datasets["train"].train_test_split(test_size=0.2)
tokenized_datasets = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"],
})

In [8]:
# Check for GPU availability
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"

In [29]:
import os

In [30]:
# Determine the number of CPU cores
num_cores = os.cpu_count()

In [31]:
print(num_cores)

12


In [32]:

# Training Arguments

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1, # Reduced for testing
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    fp16=use_cuda,
    # dataloader_num_workers=0, # Keep at 0 for debugging DataLoader issues # taking time about 68hr to finish which is good compared to last 680hr
    dataloader_num_workers=num_cores,  # Use all cores for data loading during training
    dataloader_pin_memory=use_cuda,
    max_steps=100, # Reduced for testing
)

C:\Users\kb290\anaconda3\envs\py310_torch\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
# Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [34]:
# Load model
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

OSError: The paging file is too small for this operation to complete. (os error 1455)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)



In [28]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 